# Validate Better Baseline

Found a solution at 70.627569 - need to verify it has no overlaps.

In [1]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely import affinity
from shapely.ops import unary_union

# Tree shape definition (closed polygon)
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125, 0]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5, 0.8]

def get_tree_polygon(x, y, angle_deg):
    coords = list(zip(TX, TY))
    poly = Polygon(coords)
    poly = affinity.rotate(poly, angle_deg, origin=(0, 0))
    poly = affinity.translate(poly, xoff=x, yoff=y)
    return poly

def has_overlap(poly1, poly2, tolerance=1e-9):
    if not poly1.intersects(poly2):
        return False
    intersection = poly1.intersection(poly2)
    return intersection.area > tolerance

def check_overlaps_for_n(polys):
    for i in range(len(polys)):
        for j in range(i+1, len(polys)):
            if has_overlap(polys[i], polys[j]):
                return True
    return False

def get_side_length(polys):
    union = unary_union(polys)
    bounds = union.bounds
    return max(bounds[2] - bounds[0], bounds[3] - bounds[1])

def parse_submission(csv_path):
    df = pd.read_csv(csv_path)
    df['x_val'] = df['x'].astype(str).str.replace('s', '').astype(float)
    df['y_val'] = df['y'].astype(str).str.replace('s', '').astype(float)
    df['deg_val'] = df['deg'].astype(str).str.replace('s', '').astype(float)
    df['n'] = df['id'].apply(lambda x: int(str(x).split('_')[0]))
    result = {}
    for n in range(1, 201):
        group = df[df['n'] == n]
        if len(group) != n:
            return None
        result[n] = [(row['x_val'], row['y_val'], row['deg_val']) for _, row in group.iterrows()]
    return result

print('Functions defined')

Functions defined


In [2]:
# Load and validate the better solution
better_path = '/home/nonroot/snapshots/santa-2025/21329069570/code/code/solutions/submission_70.627569.csv'
sol = parse_submission(better_path)

# Calculate score and check overlaps
total_score = 0
overlap_count = 0
overlap_ns = []

for n in range(1, 201):
    trees = sol[n]
    polys = [get_tree_polygon(x, y, deg) for x, y, deg in trees]
    side = get_side_length(polys)
    score = side**2 / n
    total_score += score
    
    if check_overlaps_for_n(polys):
        overlap_count += 1
        overlap_ns.append(n)
        if overlap_count <= 10:
            print(f"OVERLAP at N={n}")

print(f"\nTotal score: {total_score:.6f}")
print(f"Overlaps: {overlap_count}")
print(f"Overlap N values: {overlap_ns[:20]}")


Total score: 70.627569
Overlaps: 0
Overlap N values: []


In [3]:
# If no overlaps, copy to submission
import shutil
import json
import os

if overlap_count == 0:
    os.makedirs('/home/code/experiments/003_better_baseline', exist_ok=True)
    shutil.copy(better_path, '/home/code/experiments/003_better_baseline/submission.csv')
    shutil.copy(better_path, '/home/submission/submission.csv')
    
    metrics = {
        'cv_score': total_score,
        'overlaps': overlap_count,
        'source': better_path
    }
    with open('/home/code/experiments/003_better_baseline/metrics.json', 'w') as f:
        json.dump(metrics, f, indent=2)
    
    print(f"Copied to submission. Score: {total_score:.6f}")
else:
    print(f"Solution has {overlap_count} overlaps - cannot use")

Copied to submission. Score: 70.627569
